In [25]:
root_path = '/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/sy/'

In [26]:
import numpy as np
import pickle
import pandas as pd
import statsmodels.api as sm
import hdf5storage
from scipy.signal import butter, lfilter
import os, glob, time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from datetime import datetime
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import random

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car
    
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            for i in range(nChannel):
                Epochs[j, i, :] = np.subtract(Epochs[j, i, :], np.mean(eegData[i,Time_after[j]:Time_base[j]]))
        return [Epochs, Num[0]]

def Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum):
        FeaturesT = np.zeros((NumT, featureNum))
        for i in range(NumT):
            FeaturesT[i,:] = np.reshape(EpochsT[i,:,:],(1,featureNum))
        FeaturesN = np.zeros((NumN, featureNum))
        for j in range(NumN):
            FeaturesN[j,:] = np.reshape(EpochsN[j,:,:],(1,featureNum))
        return [FeaturesT,FeaturesN] 
    
def main():
        start = time.time()
        
        ##Generate Preprocessing Training data
        ctime = datetime.today().strftime("%m%d_%H%M")
        Classifier_path = '/Users/hyuns/Desktop/LDAModel/' + ctime + 'Classifier.pickle'
        channelNum = 7

        mat_path = root_path + 'Training/'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime)
        
        matfile_name = current_list[0]
        
        mat = hdf5storage.loadmat(matfile_name)
        channelNames = mat['channelNames']
        eegData = mat['eegData']
        samplingFreq = mat['samplingFreq']
        samplingFreq = samplingFreq[0,0]
        stims = mat['stims']
        channelNum = channelNames.shape
        channelNum = channelNum[1]
        eegData = np.transpose(eegData)
        
        ##Preprocessing process
        sampleNum = eegData.shape[1]
        
        #Common Average Reference
        eegData = Re_referencing(eegData, channelNum, sampleNum)

        #Bandpass Filter
        eegData = butter_bandpass_filter(eegData, 0.5, 10, samplingFreq, order=4)
    
        #Epoching
        epochSampleNum = int(np.floor(0.4 * samplingFreq))
        offset = int(np.floor(0.2 * samplingFreq)) 
        baseline = int(np.floor(0.6 * samplingFreq)) 
        [EpochsT, NumT] = Epoching(eegData, stims, 1, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        [EpochsN, NumN] = Epoching(eegData, stims, 0, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        
        #Convert to feature vector
        featureNum = channelNum*epochSampleNum
        [FeaturesT, FeaturesN] = Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum)
        TrainData = np.concatenate((FeaturesT, FeaturesN))
        TrainLabel = np.concatenate((np.ones((NumT,1)).astype(int),np.zeros((NumN ,1)).astype(int))).ravel()
        
        #Saving LDA classifier
        lda = LinearDiscriminantAnalysis(solver='lsqr',shrinkage='auto')
        lda.fit(TrainData, TrainLabel)
        joblib.dump(lda, Classifier_path, protocol=2)
        
        print("NumT :", NumT)
        print("NumN :", NumN)
        print("time :", time.time() - start)
        
if __name__ == "__main__":
    main()

NumT : 210
NumN : 1260
time : 0.5763671398162842


In [27]:
import numpy as np
from scipy.signal import butter, lfilter, sosfiltfilt
import time
import os, glob
import hdf5storage
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.externals import joblib
import shutil
from datetime import datetime
import socket
import matplotlib.pyplot as plt

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y   
    
def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            for i in range(nChannel):
                Epochs[j, i, :] = np.subtract(Epochs[j, i, :], np.mean(eegData[i,Time_after[j]:Time_base[j]]))
        
        return [Epochs, Num[0]]

def Online_Convert_to_featureVector2(Epochs, Num, featureNum):
    Features = np.zeros((Num, featureNum))
    
    for i in range(Num):
        Features[i,:] = np.reshape(Epochs[i,:,:], (1, featureNum))
    return Features


def main():
        Classifier_path = '/Users/hyuns/Desktop/LDAModel/'
        classifier_list = sorted(glob.glob(Classifier_path + '*.pickle'), key=os.path.getmtime, reverse=True)
        print("classifer:", classifier_list[0])
        
        lda = joblib.load(classifier_list[0])
        
        mat_path = root_path + 'Online/'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime, reverse=True)
        score = 0
        
        for mat_file in current_list:
            print(mat_file)
            ans = mat_file[-5]
            
            mat = hdf5storage.loadmat(mat_file)
            channelNames = mat['channelNames']
            eegData = mat['eegData']
            samplingFreq = mat['samplingFreq']
            samplingFreq = samplingFreq[0,0]
            stims = mat['stims']
            channelNum = channelNames.shape
            channelNum = channelNum[1]
            eegData = np.transpose(eegData)
            buttonNum = 7
            
            sampleNum = eegData.shape[1]
            #Common Average Reference
            eegData = Re_referencing(eegData, channelNum, sampleNum)
            
            #Bandpass Filter
            eegData = butter_bandpass_filter(eegData, 0.5, 10, samplingFreq, order=4)

            #Epoching
            epochSampleNum = int(np.floor(0.4 * samplingFreq))
            offset = int(np.floor(0.2 * samplingFreq)) 
            baseline = int(np.floor(0.6 * samplingFreq)) 
            [Epochs1, Num1] = Epoching(eegData, stims, 1, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            [Epochs2, Num2] = Epoching(eegData, stims, 2, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            [Epochs3, Num3] = Epoching(eegData, stims, 3, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            [Epochs4, Num4] = Epoching(eegData, stims, 4, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            [Epochs5, Num5] = Epoching(eegData, stims, 5, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            [Epochs6, Num6] = Epoching(eegData, stims, 6, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            [Epochs7, Num7] = Epoching(eegData, stims, 7, samplingFreq, channelNum, epochSampleNum, offset, baseline)
            
            result = np.zeros((1,buttonNum))
            featureNum = channelNum*epochSampleNum
            
            Epochs1 = Online_Convert_to_featureVector2(Epochs1, Num1, featureNum)
            Epochs2 = Online_Convert_to_featureVector2(Epochs2, Num1, featureNum)
            Epochs3 = Online_Convert_to_featureVector2(Epochs3, Num1, featureNum)
            Epochs4 = Online_Convert_to_featureVector2(Epochs4, Num1, featureNum)
            Epochs5 = Online_Convert_to_featureVector2(Epochs5, Num1, featureNum)
            Epochs6 = Online_Convert_to_featureVector2(Epochs6, Num1, featureNum)
            Epochs7 = Online_Convert_to_featureVector2(Epochs7, Num1, featureNum)
            
            a1 = lda.predict(Epochs1)
            a2 = lda.predict(Epochs2)
            a3 = lda.predict(Epochs3)
            a4 = lda.predict(Epochs4)
            a5 = lda.predict(Epochs5)
            a6 = lda.predict(Epochs6)
            a7 = lda.predict(Epochs7)

            result[0,0] = np.sum(a1)
            result[0,1] = np.sum(a2)
            result[0,2] = np.sum(a3)
            result[0,3] = np.sum(a4)
            result[0,4] = np.sum(a5)
            result[0,5] = np.sum(a6)
            result[0,6] = np.sum(a7)
            
            answer = np.argmax(result) + 1
            if int(ans) == int(answer): 
                score = score + 1
            
            print(result)
            print('order: ', ans, 'predict: ', answer)
        
        print('score:', score)
if __name__ == "__main__":
    main()

classifer: /Users/hyuns/Desktop/LDAModel/0811_1350Classifier.pickle
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/sy/Online/1_3.mat
[[0. 0. 0. 0. 0. 0. 0.]]
order:  3 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/sy/Online/2_7.mat
[[0. 0. 0. 0. 0. 0. 0.]]
order:  7 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/sy/Online/3_1.mat
[[0. 0. 0. 0. 0. 0. 0.]]
order:  1 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/sy/Online/4_2.mat
[[1. 1. 0. 0. 0. 0. 1.]]
order:  2 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/sy/Online/5_5.mat
[[0. 0. 0. 0. 0. 0. 0.]]
order:  5 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/sy/Online/6_3.mat
[[1. 1. 0. 0. 0. 0. 0.]]
order:  3 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300